# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------

from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest

# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

<h2>Preparing the data:</h2>

The first stage would be tweaking the text, following the clean -> vectorize -> transform pattern.

<h3>Cleaning the text:</h3>

- The function **clean_data** cleans the text by removing digits, spaces, whitespaces and alphanumeric characters.
- The function **clean_trainset** applies the previous function to the '**story** column, and converts the **'gender'** column to categorical values (0 for female, 1 for male).

In [10]:
def clean_data(data):
    data = re.sub(r'\w*\d+\w*', '', data) # Remove digits and numbers
    data = re.sub(r'[^\w\s]', '', data) # Remove anything that is not word character or whitespace
    data = re.sub(r'\s+', ' ', data) # Remove extra space
    data = data.strip()
    return data

In [11]:
def clean_trainset(df_train):
    
    #Create a copy
    df_train_cleaned = df_train.copy()    
    
    #Apply cleaning to the 'story' column
    df_train_cleaned['story'] = df_train_cleaned['story'].apply(clean_data)
    
    #Convert the 'gender' column into categorical values
    df_train_cleaned['gender'] = df_train_cleaned['gender'].replace(['f','m'],[0,1])
    
    return df_train_cleaned

In [12]:
#Cleaning and testing the data
df_train_cleaned = clean_trainset(df_train)

df_train_cleaned.head(8)
df_train_cleaned.shape

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,1
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,1
4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,0
5,לפני חודשים טסתי לרומא למשך שבוע טסתי במטוס אל...,0
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,1
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב הדירה הי...,0


(753, 2)

<h3>Vectorizing and transforming the text:</h3>

- The function **tokenize_text** splits the given (cleaned) text into a list of strings. It will be used as a tokenizer in the vectorization function.

- The function **set_vectorizer** sets the vectorizer object according to the given parameters.

- The function **scale_transform_trainset** transforms the trainset into a feature matrix and reduces features by using **SelectKBest**.


In [13]:
def tokenize_text(text):
    return text.split()

In [14]:
def set_vectorizer(vec_type, ngram_range, min_df):
            
    # Set the vectorizer according to the given parameters
    if vec_type == 'CountVectorizer':
        vectorizer = CountVectorizer(tokenizer = tokenize_text, min_df = min_df)
    if vec_type == 'TfidfVectorizer':
        vectorizer = TfidfVectorizer(tokenizer = tokenize_text, min_df = min_df, ngram_range = ngram_range)
    
    return vectorizer

In [15]:
def scale_transform_trainset(X_train, vectorizer, k):
    
    # Vectorize the data with the vectorizer object
    X_train_vect = vectorizer.fit_transform(X_train)
    
    #Reduce features with SelectKBest
    X_train_reduced = SelectKBest(k = k).fit_transform(X_train_vect, df_train_cleaned['gender'])
    
    return X_train_reduced

In [16]:
# Vectorizing and transforming the data

vectorizer = set_vectorizer('TfidfVectorizer', (1,1), 5)
df_train_scaled = scale_transform_trainset(df_train_cleaned['story'], vectorizer, 5000)

df_train_scaled
df_train_scaled.shape

<753x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 115475 stored elements in Compressed Sparse Row format>

(753, 5000)

<h2>Evaluating performance with cross-validation:</h2>

The function **evaluate_performance** takes the models, X_train and y_train as arguments and computes the accuracy based on K fold cross-validation.

In [17]:
def evaluate_performance(models, X_train, y_train):
    
    score = {}
    
    for model_name, model in models.items():
        # Evaluate scores by using cross-validation
        scores = cross_val_score(model, X_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10)

        trained_model = model.fit(X_train , y_train)
        y_pred = trained_model.predict(X_train)
    
        # Compute f1 scores
        f1_f = f1_score(y_train, y_pred, pos_label=0)
        f1_m = f1_score(y_train, y_pred, pos_label=1)
        f1_avg = (f1_m + f1_f)/2
   
        # Print the results
        print('Scores for model :', (model))
        print(scores)
        print('Accuracy: %.3f' % (scores.mean()))
        print('Standard deviation: %.3f\n' % (scores.std()))
        print('F1 Score (f): %.3f' % (f1_f))
        print('F1 Score (m): %.3f' % (f1_m))
        print('F1 Score (average): %.3f\n\n' % (f1_avg))
        score[model_name] = scores.mean()
    
    return score

<h3>The models:</h3>

I chose to test the following models:

- **Logistic Regression**
- **Stochasic Gradient Descent (SGD)**
- **LinearSVC**
- **Perceptron**
- **Artificial Neural Network**
- **Multinomial Naive-Bayes**
- **KNeighbors**
- **Decision Tree**

In [18]:
# Define Models for evaluation

models = {    
    'LogisticRegression': linear_model.LogisticRegression(),    
    'SGDClassifier': SGDClassifier(),
    'LinearSVC': LinearSVC(),
    'Perceptron': Perceptron(),
    'MLPClassifier': MLPClassifier(),
    'MultinomialNB': MultinomialNB(),
    'KNeighbors': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier()
}

<h3> First evaluation (5000 features):</h3>

With a relatively minimal feature removal (down from the original vectorization output of ~6000), the first evaluation reached decent accuracies for most of the models.


In [19]:
# Evaluate model performance
scores_1a = evaluate_performance(models, df_train_scaled, df_train_cleaned['gender'])

Scores for model : LogisticRegression()
[0.43283582 0.43283582 0.43283582 0.43609023 0.43609023 0.43181818
 0.43181818 0.43181818 0.43181818 0.43181818]
Accuracy: 0.433
Standard deviation: 0.002

F1 Score (f): 0.055
F1 Score (m): 0.869
F1 Score (average): 0.462


Scores for model : SGDClassifier()
[0.75081967 0.80208333 0.87540984 0.79166667 0.67431762 0.73653396
 0.72084367 0.688      0.76489028 0.79674797]
Accuracy: 0.760
Standard deviation: 0.057

F1 Score (f): 0.994
F1 Score (m): 0.998
F1 Score (average): 0.996


Scores for model : LinearSVC()
[0.72161172 0.70725926 0.703125   0.75158085 0.70238095 0.70238095
 0.6662102  0.62713068 0.65277778 0.6662102 ]
Accuracy: 0.690
Standard deviation: 0.035

F1 Score (f): 0.994
F1 Score (m): 0.998
F1 Score (average): 0.996


Scores for model : Perceptron()
[0.72301654 0.73207991 0.62797203 0.75158085 0.69262295 0.74747475
 0.67431762 0.77636216 0.74747475 0.88590264]
Accuracy: 0.736
Standard deviation: 0.065

F1 Score (f): 0.975
F1 Score (m): 

<h2>Finding the best hyperparameters:</h2>

By using the grid search, the following function will attempt to find the best parameters.

<h3> Defining hyperparameters:</h3>

In [20]:
# Create a dictionary with hyperparameters for tuning
params = {    
    'LogisticRegression': { 'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2', None]},      
    'SGDClassifier': { 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                            'penalty': ['l2', 'l1', 'elasticnet'],
                            'alpha': [0.0001, 0.05]},
    'LinearSVC': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'dual': [False]},
    'Perceptron': {'alpha': [0.0001, 0.05], 'penalty': [None, 'l2', 'l1', 'elasticnet']},
    'MLPClassifier': {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                            'activation': ['tanh', 'relu'],
                            'solver': ['sgd', 'adam'],
                            'alpha': [0.0001, 0.05],
                            'learning_rate': ['constant','adaptive']}, 
    'MultinomialNB': {'fit_prior': [True, False]},
    'KNeighbors': {'n_neighbors': [3, 5, 10], 'weights': ['uniform', 'distance'] },
    'DecisionTree': {'criterion': ['gini', 'entropy'],
                     'max_depth': [1, 2, 5, 7, 10],
                     'min_samples_split': [5, 10, 15, 20, 25],
                     'min_samples_leaf': [1, 2, 3, 4, 5]}
}

# Create a dictionary holding the models with parameters
models_params = dict()
models_params['LogisticRegression'] = linear_model.LogisticRegression(**params['LogisticRegression']) 
models_params['SGDClassifier'] = SGDClassifier(**params['SGDClassifier'])
models_params['LinearSVC'] = LinearSVC(**params['LinearSVC'])
models_params['Perceptron'] = Perceptron(**params['Perceptron'])
models_params['MLPClassifier'] = MLPClassifier(**params['MLPClassifier'])
models_params['MultinomialNB'] = MultinomialNB(**params['MultinomialNB'])
models_params['KNeighbors'] = KNeighborsClassifier(**params['KNeighbors'])
models_params['DecisionTree'] = DecisionTreeClassifier(**params['DecisionTree'])

<h3> Finding the best hyperparameters:</h3>

In [21]:
def find_best_params(models, params, X_train, y_train):
        
    for model_name, model in models_params.items():
        print("Finding best hyperparameters for:", model_name)
        grid = GridSearchCV(model, params[model_name], cv=5, n_jobs=-1)
        grid.fit(X_train, y_train)        
        print("Best parameters:", grid.best_params_)
        print("\n\n")

In [22]:
find_best_params(models, params, df_train_scaled, df_train_cleaned['gender'])

Finding best hyperparameters for: LogisticRegression
Best parameters: {'C': 0.01, 'penalty': None}



Finding best hyperparameters for: SGDClassifier
Best parameters: {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2'}



Finding best hyperparameters for: LinearSVC
Best parameters: {'C': 100, 'dual': False, 'penalty': 'l2'}



Finding best hyperparameters for: Perceptron
Best parameters: {'alpha': 0.0001, 'penalty': 'l2'}



Finding best hyperparameters for: MLPClassifier
Best parameters: {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}



Finding best hyperparameters for: MultinomialNB
Best parameters: {'fit_prior': False}



Finding best hyperparameters for: KNeighbors
Best parameters: {'n_neighbors': 3, 'weights': 'distance'}



Finding best hyperparameters for: DecisionTree
Best parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 25}





In [23]:
models_best_params = {    
    'LogisticRegression': linear_model.LogisticRegression(C = 0.01, penalty = None),    
    'SGDClassifier': SGDClassifier(alpha = 0.0001, loss = 'hinge', penalty = 'elasticnet'),
    'LinearSVC': LinearSVC(C = 100, dual = False, penalty = 'l2'),
    'Perceptron': Perceptron(alpha = 0.0001, penalty = None),
    'MLPClassifier': MLPClassifier(activation = 'relu', alpha = 0.05, hidden_layer_sizes = (50, 100, 50), learning_rate = 'constant', solver = 'adam'),
    'MultinomialNB': MultinomialNB(fit_prior = False),
    'KNeighbors': KNeighborsClassifier(n_neighbors = 3, weights = 'distance'),
    'DecisionTree': DecisionTreeClassifier(criterion = 'entropy', max_depth = 5, min_samples_leaf = 2, min_samples_split = 25)
}

scores_1b = evaluate_performance(models_best_params, df_train_scaled, df_train_cleaned['gender'])

Scores for model : LogisticRegression(C=0.01, penalty=None)
[0.70725926 0.73660357 0.76562938 0.77636216 0.75       0.73570073
 0.79166667 0.592      0.80137712 0.8076375 ]
Accuracy: 0.746
Standard deviation: 0.060

F1 Score (f): 0.997
F1 Score (m): 0.999
F1 Score (average): 0.998


Scores for model : SGDClassifier(penalty='elasticnet')
[0.72301654 0.82582124 0.73669291 0.77636216 0.67431762 0.70641373
 0.70238095 0.69262295 0.75       0.76736973]
Accuracy: 0.735
Standard deviation: 0.043

F1 Score (f): 0.994
F1 Score (m): 0.998
F1 Score (average): 0.996


Scores for model : LinearSVC(C=100, dual=False)
[0.72161172 0.82582124 0.76800977 0.80137712 0.70641373 0.73653396
 0.688      0.688      0.75       0.79674797]
Accuracy: 0.748
Standard deviation: 0.046

F1 Score (f): 0.997
F1 Score (m): 0.999
F1 Score (average): 0.998


Scores for model : Perceptron()
[0.72301654 0.73207991 0.62797203 0.75158085 0.69262295 0.74747475
 0.67431762 0.77636216 0.74747475 0.88590264]
Accuracy: 0.736
Stan

<h3> Second evaluation (1000 features):</h3>

In the second evaluation, I tested a slightly smaller feature count of 1000, which improved most of the scores. By using the same hyperparameter setting from the previous evaluation, I was able to further improve the accuracy for some of them, most notably the Logistic Regression model.


In [24]:
df_train_scaled2 = scale_transform_trainset(df_train_cleaned['story'], vectorizer, 1000)

scores_2a = evaluate_performance(models, df_train_scaled2, df_train_cleaned['gender'])

Scores for model : LogisticRegression()
[0.43283582 0.43283582 0.43283582 0.43609023 0.43609023 0.43181818
 0.43181818 0.43181818 0.43181818 0.43181818]
Accuracy: 0.433
Standard deviation: 0.002

F1 Score (f): 0.000
F1 Score (m): 0.866
F1 Score (average): 0.433


Scores for model : SGDClassifier()
[0.81764863 0.86452763 0.92083333 0.92055085 0.92055085 0.8076375
 0.8076375  0.84110169 0.88082627 0.89834644]
Accuracy: 0.868
Standard deviation: 0.045

F1 Score (f): 0.992
F1 Score (m): 0.997
F1 Score (average): 0.994


Scores for model : LinearSVC()
[0.76800977 0.81764863 0.79733333 0.81389578 0.784      0.76736973
 0.736      0.688      0.875      0.82435597]
Accuracy: 0.787
Standard deviation: 0.049

F1 Score (f): 0.915
F1 Score (m): 0.976
F1 Score (average): 0.946


Scores for model : Perceptron()
[0.91377354 0.83924794 0.92422732 0.89035088 0.94409938 0.85902256
 0.79166667 0.92951128 0.89035088 0.92393509]
Accuracy: 0.891
Standard deviation: 0.045

F1 Score (f): 0.975
F1 Score (m): 0

In [25]:
scores_2b = evaluate_performance(models_best_params, df_train_scaled2, df_train_cleaned['gender'])

Scores for model : LogisticRegression(C=0.01, penalty=None)
[0.89473684 0.88634098 0.94194041 0.96027542 0.94172494 0.86402486
 0.83333333 0.80137712 0.92393509 0.94172494]
Accuracy: 0.899
Standard deviation: 0.050

F1 Score (f): 0.997
F1 Score (m): 0.999
F1 Score (average): 0.998


Scores for model : SGDClassifier(penalty='elasticnet')
[0.81764863 0.86452763 0.89869368 0.92690058 0.92055085 0.83333333
 0.78044496 0.81702358 0.85768501 0.89834644]
Accuracy: 0.862
Standard deviation: 0.047

F1 Score (f): 0.992
F1 Score (m): 0.997
F1 Score (average): 0.994


Scores for model : LinearSVC(C=100, dual=False)
[0.80208333 0.86452763 0.92422732 0.88590264 0.92055085 0.81702358
 0.78044496 0.86402486 0.88590264 0.89834644]
Accuracy: 0.864
Standard deviation: 0.047

F1 Score (f): 0.997
F1 Score (m): 0.999
F1 Score (average): 0.998


Scores for model : Perceptron()
[0.91377354 0.83924794 0.92422732 0.89035088 0.94409938 0.85902256
 0.79166667 0.92951128 0.89035088 0.92393509]
Accuracy: 0.891
Stan

<h3> Analyzing the results:</h3>

Some of the models were improved, but some scores became lower. Therefore, I attempted to manually tune them. Though unlike the case of Logistic Regression, which improved significantly, some of the models, such as MultinomialNB and KNeighbors, only yielded marginal increases and their accuracy remained pretty low.

Obviously, reducing the number of features helped improve the models' accuracies, reaching scores that ranged around ~0.8, but the loss of data probably makes them less realistic, as less frequent words are to be expected in texts, and they affect predictability.

For my final prediction model, I chose to omit the models that did not score well, and remained with the following settings:

- **LogisticRegression(C=0.01, penalty=None)**
- **SGDClassifier()**
- **LinearSVC(C=100, dual=False)**
- **Perceptron()**
- **MLPClassifier()**

In [26]:
df_comp = pd.DataFrame(scores_1a.items(), columns=['model', '5000 without tuning'])
df_comp['5000 with tuning'] = df_comp['model'].map(scores_1b)
df_comp['1000 without tuning'] = df_comp['model'].map(scores_2a)
df_comp['1000 with tuning'] = df_comp['model'].map(scores_2b)

df_comp["best result (5000)"] = df_comp[["5000 with tuning", "5000 without tuning"]].max(axis=1)
df_comp["best result (1000)"] = df_comp[["1000 with tuning", "1000 without tuning"]].max(axis=1)

df_comp

,model,5000 without tuning,5000 with tuning,1000 without tuning,1000 with tuning,best result (5000),best result (1000)
0,LogisticRegression,0.433,0.746,0.433,0.899,0.746,0.899
1,SGDClassifier,0.760,0.735,0.868,0.862,0.760,0.868
2,LinearSVC,0.690,0.748,0.787,0.864,0.748,0.864
3,Perceptron,0.736,0.736,0.891,0.891,0.736,0.891
4,MLPClassifier,0.728,0.732,0.911,0.863,0.732,0.911
5,MultinomialNB,0.433,0.450,0.433,0.757,0.450,0.757
6,KNeighbors,0.449,0.510,0.439,0.466,0.510,0.466
7,DecisionTree,0.615,0.514,0.617,0.518,0.615,0.617


<h2>Predicting:</h2>

The function **predict_dataset** will apply the models in the previous stage to and try and predict the gender of the story's writers.

In [27]:
def predict_dataset(models, X, y, df_test):
    
    # Create the prediction dataframe    
    df_predicted = df_test[['story']].copy()
    X_test =  df_test['story'].copy()
    
    # Apply cleaning, vectorization and scaling
    X_test = X_test.apply(clean_data)
    vect_pred = set_vectorizer('TfidfVectorizer', (1,1), 5)    
    X_test = vect_pred.fit_transform(X_test)    
    n = X_test.shape[1] #Due to time constraints I had to manually select only n columns for the models to match.
        
    # For every model, create a new column containing its predictions, converted back to 'f'/'m'.
    for model_name, model in models.items():
        model.fit(X.iloc[:, :n], y)
        y_pred = model.predict(X_test)        
        df_predicted[model_name] = y_pred
        df_predicted[model_name] = df_predicted[model_name].replace([0,1],['f','m'])
        print('Predictions for model: ', model)
        print(df_predicted[model_name].value_counts())
        print('\n') 
    
    return df_predicted

In [28]:
# Define models for prediction
models_pred = dict()
models_pred['LogisticRegression'] = linear_model.LogisticRegression(C=0.01, penalty=None) 
models_pred['SGDClassifier'] = SGDClassifier()
models_pred['LinearSVC'] = LinearSVC(C=100, dual=False)
models_pred['Perceptron'] = Perceptron()
models_pred['MLPClassifier'] = MLPClassifier()

X = pd.DataFrame(df_train_scaled.toarray())

df_predicted = predict_dataset(models_pred, X, df_train_cleaned['gender'], df_test)

Predictions for model:  LogisticRegression(C=0.01, penalty=None)
m    297
f     26
Name: LogisticRegression, dtype: int64


Predictions for model:  SGDClassifier()
m    309
f     14
Name: SGDClassifier, dtype: int64


Predictions for model:  LinearSVC(C=100, dual=False)
m    289
f     34
Name: LinearSVC, dtype: int64


Predictions for model:  Perceptron()
m    320
f      3
Name: Perceptron, dtype: int64


Predictions for model:  MLPClassifier()
m    320
f      3
Name: MLPClassifier, dtype: int64




In [29]:
# Display the first and last 5 entries and their predictions
df_predicted.head(5)
df_predicted.tail(5)

,story,LogisticRegression,SGDClassifier,LinearSVC,Perceptron,MLPClassifier
0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m,m,m,m,m
1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m,m,m,m,m
2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m,m,m,m,m
3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m,m,m,m,m
4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m,m,f,m,m


,story,LogisticRegression,SGDClassifier,LinearSVC,Perceptron,MLPClassifier
318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m,m,m,m,m
319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m,m,m,m,m
320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m,m,m,m,m
321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m,m,m,m,m
322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m,m,m,m,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [30]:
df_predicted.to_csv('classification_results.csv',index=False)